### Modified binary search

#### Order agnostic binary search
List is sorted, asc or desc

In [1]:
# detect the order, update mid accordingly, repeat till found

def binary_search(arr, key):
    start, end = 0, len(arr) - 1
    isAscending = arr[start] < arr[end]
    
    while start <= end:
        # calculate the middle of the current range
        mid = start + (end - start) // 2

        if key == arr[mid]:
            return mid

        if isAscending:  # ascending order
            if key < arr[mid]:
                end = mid - 1  # the 'key' can be in the first half
            else:  # key > arr[mid]
                start = mid + 1  # the 'key' can be in the second half
        else:  # descending order
            if key > arr[mid]:
                end = mid - 1  # the 'key' can be in the first half
            else:  # key < arr[mid]
                start = mid + 1  # the 'key' can be in the second half

    return -1  # element not found

In [2]:
print (binary_search([1,2,3,4,5], 4))
print (binary_search([9,8,7,6,5,4], 4))

3
5


#### Ceiling number in a sorted list
([1, 3, 5, 6, 7, 9], 4) => 5

In [3]:
# trivial binary search, return mid+1 th index

def search_ceiling_of_a_number(arr, key):
    n = len(arr)
    if key > arr[n - 1]:  # if the 'key' is bigger than the biggest element
        return -1

    start, end = 0, n - 1
    
    while start <= end:
        mid = start + (end - start) // 2
        if key < arr[mid]:
            end = mid - 1
        elif key > arr[mid]:
            start = mid + 1
        else:  # found the key
            return mid

    # since the loop is running until 'start <= end', so at the end of the while loop, 
    # 'start == end+1' we are not able to find the element in the given array, so the next
    # big number will be arr[start]
    return start

In [4]:
print (search_ceiling_of_a_number([1,2,3,5,6,7], 4))

3


#### Next letter, list of chars, circular list
* (['a', 'c', 'f', 'h'], 'f') => 'h'
* (['a', 'c', 'f', 'h'], 'm') => 'a'
* (['a', 'c', 'f', 'h'], 'h') => 'a'

In [5]:
# circular list handled via %, rest is similar to ceiling number above

def search_next_letter(letters, key):
    n = len(letters)
    start, end = 0, n - 1
    
    while start <= end:
        mid = start + (end - start) // 2
        if key < letters[mid]:
            end = mid - 1
        else: # key >= letters[mid]:
            start = mid + 1

    # since the loop is running until 'start <= end', so at the end of the while loop, 
    # 'start == end+1'
    return letters[start % n]

In [6]:
print (search_next_letter(['a', 'c', 'f', 'h'], 'h'))

a


#### Number range
Return first and last occurance of given number in a sorted list
([1, 6, 6, 6, 7, 9], 6) => [1, 3]

In [7]:
# do bin search twice, first to find the first index and second to find the second index
# key idea: update start&end differently ONLY when you have found the element (how? based on weather you're searching for first index or second)


def find_range(arr, key):
    result = [- 1, -1]
    
    result[0] = binary_search(arr, key, False)
    
    if result[0] != -1:  # no need to search, if 'key' is not present in the input array
        result[1] = binary_search(arr, key, True)
        
    return result


# modified Binary Search
def binary_search(arr, key, findMaxIndex):
    keyIndex = -1
    start, end = 0, len(arr) - 1
    
    while start <= end:
        mid = start + (end - start) // 2
        if key < arr[mid]:
            end = mid - 1
        elif key > arr[mid]:
            start = mid + 1
        else:  # key == arr[mid]
            keyIndex = mid
            if findMaxIndex:
                start = mid + 1  # search ahead to find the last index of 'key'
            else:
                end = mid - 1  # search behind to find the first index of 'key'

    return keyIndex

In [8]:
print (find_range([1, 5, 5, 5, 6, 8, 9], 5))

[1, 3]


#### Bitonic array, peak finding
([1, 3, 5, 7, 6, 4, 2]) => 7

In [9]:
# key idea: comparision between mid & mid+1

def find_max_in_bitonic_array(arr):
    start, end = 0, len(arr) - 1
    
    while start < end:
        mid = start + (end - start) // 2
    
        if arr[mid] > arr[mid + 1]:
            end = mid
        else:
            start = mid + 1

    # at the end of the while loop, 'start == end'
    return arr[start]

In [11]:
print (find_max_in_bitonic_array([1, 3, 5, 7, 6, 4, 2]))

7


#### Search in infinite sorted array (stream)

In [13]:
# exponentially increase the bounds till you find the searchable bound, once found, perform binary search

import math


class ArrayReader:

    def __init__(self, arr):
        self.arr = arr

    def get(self, index):
        if index >= len(self.arr):
            return math.inf
        return self.arr[index]


def search_in_infinite_array(reader, key):
    # find the proper bounds first
    start, end = 0, 1
    
    while reader.get(end) < key:
        newStart = end + 1
        end += (end - start + 1) * 2
        # increase to double the bounds size
        start = newStart

    return binary_search(reader, key, start, end)


def binary_search(reader, key, start, end):
    while start <= end:
        mid = start + (end - start) // 2
        if key < reader.get(mid):
            end = mid - 1
        elif key > reader.get(mid):
            start = mid + 1
        else:  # found the key
            return mid

    return -1

In [14]:
reader = ArrayReader([4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30])
print(search_in_infinite_array(reader, 16))

6


#### Minimum difference element
([1, 3, 8, 10, 15], 12) => 10

In [15]:
# search the array using bin search till it fails (as the element wouldnt exist) and then look left or right + retrn min

def search_min_diff_element(arr, key):
    if key < arr[0]:
        return arr[0]
    
    n = len(arr)
    
    if key > arr[n - 1]:
        return arr[n - 1]

    start, end = 0, n - 1
    
    while start <= end:
        mid = start + (end - start) // 2
        if key < arr[mid]:
            end = mid - 1
        elif key > arr[mid]:
            start = mid + 1
        else:
            return arr[mid]

    # at the end of the while loop, 'start == end+1'
    # we are not able to find the element in the given array
    # return the element which is closest to the 'key'
    if (arr[start] - key) < (key - arr[end]):
        return arr[start]
    
    return arr[end]


In [16]:
print(search_min_diff_element([1, 3, 8, 10, 15], 12))

10
